In [20]:
import pandas as pd 
import geopandas as gpd
import sys
import os
sys.path.append(os.path.abspath("../.."))
from Search.Search_Archives import encontrar_arquivo
from Search.Search_Diretory import encontrar_diretorio


estabelecimentos = pd.read_csv('estabelecimentos_corrigidos.csv')

#estabelecimentos

path = encontrar_diretorio("distritos_quadras")
arquivo_geojson = os.path.join(path, "sao_paulo_contorno.geojson")
sp_contorno = gpd.read_file(arquivo_geojson)

#sp_contorno

# Convert estabelecimentos to a GeoDataFrame
estabelecimentos_gdf = gpd.GeoDataFrame(
    estabelecimentos,
    geometry=gpd.points_from_xy(estabelecimentos.Longitude, estabelecimentos.Latitude),
    crs=sp_contorno.crs
)

# Perform spatial join to find estabelecimentos outside of sp_contorno
estabelecimentos_outside_contorno = estabelecimentos_gdf[~estabelecimentos_gdf.geometry.within(sp_contorno.unary_union)]

estabelecimentos_outside_contorno

C:\Users\Denil\AppData\Local\Temp\ipykernel_15256\195060363.py:28: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  estabelecimentos_outside_contorno = estabelecimentos_gdf[~estabelecimentos_gdf.geometry.within(sp_contorno.unary_union)]


,id,Rede,Nome,Localizado em,Endereço,Bairro,CEP,Tipo,Latitude,Longitude,Unnamed: 10,geometry
2,Academia de Ginastica_10.395.293,Companhia Athletica,Cia111,Rua,"R José Guerra, 111",Chácara Santo Antônio,04719-030,Academia de Ginastica,-23.618596,-46.573278,NaN,POINT (-46.57328 -23.6186)
7,Academia de Ginastica_11.093.389,Smart Fit,Academia Smart Fit Sapopemba,Rua,Av. Sapopemba,Jardim Adutora,03988-010,Academia de Ginastica,-23.620798,-46.513838,NaN,POINT (-46.51384 -23.6208)
14,Academia de Ginastica_11.093.396,Smart Fit,Academia Smart Fit Agua Branca,Rua,Av. Francisco Matarazzo,Água Branca,05001-100,Academia de Ginastica,-23.633018,-46.515757,NaN,POINT (-46.51576 -23.63302)
16,Academia de Ginastica_11.093.398,Bluefit,Bluefit Continental,Rua,100 Pavmto2,Parque Continental,05328-020,Academia de Ginastica,-23.546261,-46.761435,NaN,POINT (-46.76144 -23.54626)
20,Academia de Ginastica_11.093.402,Smart Fit,Academia Smart Fit Shopping Heliopolis,Rua,Est. das Lágrimas,São João Climaco,04244-000,Academia de Ginastica,-23.412811,-46.456712,NaN,POINT (-46.45671 -23.41281)
...,...,...,...,...,...,...,...,...,...,...,...,...
13605,Shopping Center_358292,NaN,Raposo Shopping,Rua,Rod Raposo Tavares Km 145,Jd Boa Vista,05577-200,Shopping Center,-23.638568,-46.561456,NaN,POINT (-46.56146 -23.63857)
13623,Shopping Center_358321,NaN,Nações Unidas,Rua,Av das Nações Unidas 12901,Brooklin Novo,04578-910,Shopping Center,-23.524700,-46.758439,NaN,POINT (-46.75844 -23.5247)
13625,Shopping Center_358330,NaN,Vila Olímpia,Rua,R Olimpiadas 360,Vila Olimpia,04551-000,Shopping Center,-20.738465,-48.913529,NaN,POINT (-48.91353 -20.73846)
13630,Shopping Center_358597,NaN,Cidade Jardim,Rua,Av Magalhães de Castro 12000,Real Parque,05502-001,Shopping Center,-23.028230,-45.573358,NaN,POINT (-45.57336 -23.02823)


In [14]:
def formatar_cep(cep):
    # Converte o CEP para string, caso seja um número
    cep_str = str(cep)
    
    # Verifica se o CEP tem 8 dígitos
    if len(cep_str) == 8:
        # Formata o CEP no formato XXXXX-XXX
        return f"{cep_str[:5]}-{cep_str[5:]}"
    
    # Se o CEP já estiver no formato correto ou não for válido, retorna o CEP original
    return cep_str

## Ajustar pontos fora do contorno

In [19]:
import pandas as pd
import geopandas as gpd
import requests
import os
from pathlib import Path

# Configuração para o Mapbox
TOKEN = "pk.eyJ1IjoicHJvamV0b2RhZG9zIiwiYSI6ImNtMXdiNjVobTBpa2Eya3BsMnR5OWxsd3AifQ.SGh5qTES1kmMN3VNFzZAwQ"
SAO_PAULO = (-46.63389, -23.55028)

# Função para buscar coordenadas usando a API do Mapbox
def obter_coordenadas(endereco, bairro, cep):
    search_text = f"{cep} São Paulo {bairro} {endereco}"
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{search_text}.json?access_token={TOKEN}&proximity={SAO_PAULO[0]},{SAO_PAULO[1]}"
    try:
        response = requests.get(url)
        data = response.json()
        if data.get("features"):
            lat = data["features"][0]["geometry"]["coordinates"][1]
            lon = data["features"][0]["geometry"]["coordinates"][0]
            return lat, lon
    except requests.exceptions.RequestException as e:
        print(f"Erro na solicitação para o endereço {search_text}: {e}")
    return None, None

# Carregar a tabela maior
estabelecimentos = pd.read_csv('Estabelecimentos.csv')

# Carregar o contorno de São Paulo
path = encontrar_diretorio("distritos_quadras")
arquivo_geojson = os.path.join(path, "sao_paulo_contorno.geojson")
sp_contorno = gpd.read_file(arquivo_geojson)

# Converter estabelecimentos em um GeoDataFrame
estabelecimentos_gdf = gpd.GeoDataFrame(
    estabelecimentos,
    geometry=gpd.points_from_xy(estabelecimentos.Longitude, estabelecimentos.Latitude),
    crs=sp_contorno.crs
)

# Identificar pontos fora do contorno
estabelecimentos_outside_contorno = estabelecimentos_gdf[~estabelecimentos_gdf.geometry.within(sp_contorno.unary_union)]

# Tentar carregar o arquivo de correções anteriores (se existir)
if os.path.exists('estabelecimentos_corrigidos.csv'):
    df_corrigido = pd.read_csv('estabelecimentos_corrigidos.csv')
else:
    # Se o arquivo não existir, usamos o dataframe original
    df_corrigido = estabelecimentos.copy()

# Contador de linhas processadas
count = 0

# Corrigir as latitudes e longitudes erradas
for idx, row in estabelecimentos_outside_contorno.iterrows():
    endereco = row['Endereço']
    bairro = row['Bairro']
    cep_nao_formatado = row['CEP']
    cep = formatar_cep(cep_nao_formatado)
    
    # Verificar se o CEP é uma lista, e pegar o primeiro valor caso seja
    if isinstance(cep, list):
        cep = cep[0]
    
    lat, lon = obter_coordenadas(endereco, bairro, cep)

    if lat and lon:
        df_corrigido.loc[idx, 'Latitude'] = lat
        df_corrigido.loc[idx, 'Longitude'] = lon
    
    # Incrementar o contador e salvar o arquivo a cada 100 linhas processadas
    count += 1
    if count % 100 == 0:
        print(f"{count} linhas processadas. Salvando progresso...")
        df_corrigido.to_csv('estabelecimentos_corrigidos.csv', index=False)

# Salvar a tabela corrigida
df_corrigido.to_csv('estabelecimentos_corrigidos.csv', index=False)

print("Processo concluído. Arquivo salvo como 'estabelecimentos_corrigidos.csv'.")

C:\Users\Denil\AppData\Local\Temp\ipykernel_15256\2989454340.py:42: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  estabelecimentos_outside_contorno = estabelecimentos_gdf[~estabelecimentos_gdf.geometry.within(sp_contorno.unary_union)]


100 linhas processadas. Salvando progresso...
200 linhas processadas. Salvando progresso...
300 linhas processadas. Salvando progresso...
400 linhas processadas. Salvando progresso...
500 linhas processadas. Salvando progresso...
600 linhas processadas. Salvando progresso...
700 linhas processadas. Salvando progresso...
800 linhas processadas. Salvando progresso...
900 linhas processadas. Salvando progresso...
1000 linhas processadas. Salvando progresso...
1100 linhas processadas. Salvando progresso...
1200 linhas processadas. Salvando progresso...
1300 linhas processadas. Salvando progresso...
1400 linhas processadas. Salvando progresso...
1500 linhas processadas. Salvando progresso...
1600 linhas processadas. Salvando progresso...
1700 linhas processadas. Salvando progresso...
1800 linhas processadas. Salvando progresso...
1900 linhas processadas. Salvando progresso...
2000 linhas processadas. Salvando progresso...
2100 linhas processadas. Salvando progresso...
2200 linhas processada